In [192]:
import pandas as pd
import sqlite3
import json
from collections import Counter
import pandas as pd

# Getting Data

Connect to sql ```yelp.db``` and read data.

In [193]:
# establish connection to yelp sqlite db
connection = sqlite3.connect('../yelp.db')

# select all columns from the search_state table
query = 'SELECT * FROM business_search_results'

# read query into dataframe
df = pd.read_sql_query(query, connection)

connection.close()

df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,...,phone,display_phone,distance,attributes,price,_page_url,_loaded_at,_location,_term,_is_complete
0,rWBOXmdnFTnTWc1jBne0Rw,southern-n-smoked-baltimore,Southern N Smoked,https://s3-media4.fl.yelpcdn.com/bphoto/jd6s9-...,0,https://www.yelp.com/biz/southern-n-smoked-bal...,3,"[{""alias"": ""bbq"", ""title"": ""Barbeque""}, {""alia...",4.7,"{""latitude"": 39.27682860877056, ""longitude"": -...",...,+16673123102,(667) 312-3102,2219.738068,"{""business_temp_closed"": null, ""menu_url"": ""ht...",None,https://api.yelp.com/v3/businesses/search?&ter...,2024-04-23 14:23:10.750596+00:00,"Baltimore, MD 21201",Black owned,True
1,6ulR8s_SK3X0wYVQNZ0DSQ,next-phaze-cafe-baltimore,Next Phaze Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/k61Fmo...,0,https://www.yelp.com/biz/next-phaze-cafe-balti...,136,"[{""alias"": ""cafes"", ""title"": ""Cafes""}, {""alias...",4.0,"{""latitude"": 39.2913148, ""longitude"": -76.6130...",...,+14104493232,(410) 449-3232,1019.219962,"{""business_temp_closed"": null, ""menu_url"": ""ht...",$$,https://api.yelp.com/v3/businesses/search?&ter...,2024-04-23 14:23:10.750596+00:00,"Baltimore, MD 21201",Black owned,True
2,s0LPuaTzR9euWzNT52o49Q,sugarbees-cafe-and-grill-baltimore-5,Sugarbee's Cafe & Grill,https://s3-media2.fl.yelpcdn.com/bphoto/wJdfsJ...,0,https://www.yelp.com/biz/sugarbees-cafe-and-gr...,39,"[{""alias"": ""tradamerican"", ""title"": ""American""...",4.2,"{""latitude"": 39.291817, ""longitude"": -76.615902}",...,+14104430475,(410) 443-0475,741.811219,"{""business_temp_closed"": null, ""menu_url"": nul...",$,https://api.yelp.com/v3/businesses/search?&ter...,2024-04-23 14:23:10.750596+00:00,"Baltimore, MD 21201",Black owned,True
3,kTOlqpQmnA-8nWxE0lbFEA,the-land-of-kush-baltimore,The Land of Kush,https://s3-media1.fl.yelpcdn.com/bphoto/q_M4O9...,0,https://www.yelp.com/biz/the-land-of-kush-balt...,797,"[{""alias"": ""vegan"", ""title"": ""Vegan""}, {""alias...",4.1,"{""latitude"": 39.3001673, ""longitude"": -76.6216...",...,+14102255874,(410) 225-5874,608.964984,"{""business_temp_closed"": null, ""menu_url"": nul...",$$,https://api.yelp.com/v3/businesses/search?&ter...,2024-04-23 14:23:10.750596+00:00,"Baltimore, MD 21201",Black owned,True
4,_-kMik32dINn2Xas6YvRzA,the-ivy-hotel-baltimore,The Ivy Hotel,https://s3-media2.fl.yelpcdn.com/bphoto/q4q047...,0,https://www.yelp.com/biz/the-ivy-hotel-baltimo...,67,"[{""alias"": ""hotels"", ""title"": ""Hotels""}]",4.7,"{""latitude"": 39.3035166638629, ""longitude"": -7...",...,+14105146500,(410) 514-6500,1347.893495,"{""business_temp_closed"": null, ""waitlist_reser...",$$$$,https://api.yelp.com/v3/businesses/search?&ter...,2024-04-23 14:23:10.750596+00:00,"Baltimore, MD 21201",Black owned,True


# Data Cleaning

In [194]:
# remove extra columns that arent needed for analysis
columns_to_drop = ['alias', 'image_url', 'is_closed', 'url', 'review_count', 'rating', 
    'transactions', 'phone', 'display_phone', 'distance', 'attributes', 'price',
    '_page_url', '_loaded_at', '_term', '_is_complete', '_location' # metadata
    ]
    
df.drop(columns=columns_to_drop, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20290 entries, 0 to 20289
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           20290 non-null  object
 1   name         20290 non-null  object
 2   categories   20290 non-null  object
 3   coordinates  20290 non-null  object
 4   location     20290 non-null  object
dtypes: object(5)
memory usage: 792.7+ KB


In [195]:
# change to relevant datatypes
df['id'] = df['id'].astype(str)
df['name'] = df['name'].astype(str)
df['categories'] = df['categories'].apply(json.loads)
df['coordinates'] = df['coordinates'].apply(json.loads)
df['location'] = df['location'].apply(json.loads)

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20290 entries, 0 to 20289
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           20290 non-null  object
 1   name         20290 non-null  object
 2   categories   20290 non-null  object
 3   coordinates  20290 non-null  object
 4   location     20290 non-null  object
dtypes: object(5)
memory usage: 792.7+ KB


In [197]:
# extract latitude and longitude and format to 6 decimal places as per google looker studio's map requirements
df['coordinates'] = df['coordinates'].apply(lambda x: '{:.6f}, {:.6f}'.format(float(x['latitude']), float(x['longitude'])))
# df['coordinates'] = df['coordinates'].apply(lambda x: '{:.6f}'.format(float(x.split(',')[0])) + ', {:.6f}'.format(float(x.split(',')[1])))

# extract zip code from location
df['zip_code'] = df['location'].apply(lambda x: x['zip_code'])

df.drop(columns=['location'], inplace=True)

df.head()

,id,name,categories,coordinates,zip_code
0,rWBOXmdnFTnTWc1jBne0Rw,Southern N Smoked,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...","39.276829, -76.613247",21230
1,6ulR8s_SK3X0wYVQNZ0DSQ,Next Phaze Cafe,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","39.291315, -76.613020",21202
2,s0LPuaTzR9euWzNT52o49Q,Sugarbee's Cafe & Grill,"[{'alias': 'tradamerican', 'title': 'American'...","39.291817, -76.615902",21201
3,kTOlqpQmnA-8nWxE0lbFEA,The Land of Kush,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...","39.300167, -76.621694",21201
4,_-kMik32dINn2Xas6YvRzA,The Ivy Hotel,"[{'alias': 'hotels', 'title': 'Hotels'}]","39.303517, -76.612873",21202


In [198]:
# Remove rows with duplicate ids
df = df.drop_duplicates(subset=['id'])

df.head()

,id,name,categories,coordinates,zip_code
0,rWBOXmdnFTnTWc1jBne0Rw,Southern N Smoked,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...","39.276829, -76.613247",21230
1,6ulR8s_SK3X0wYVQNZ0DSQ,Next Phaze Cafe,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...","39.291315, -76.613020",21202
2,s0LPuaTzR9euWzNT52o49Q,Sugarbee's Cafe & Grill,"[{'alias': 'tradamerican', 'title': 'American'...","39.291817, -76.615902",21201
3,kTOlqpQmnA-8nWxE0lbFEA,The Land of Kush,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...","39.300167, -76.621694",21201
4,_-kMik32dINn2Xas6YvRzA,The Ivy Hotel,"[{'alias': 'hotels', 'title': 'Hotels'}]","39.303517, -76.612873",21202


In [199]:
# extract category titles to help indicate industry
df['categories'] = df['categories'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# df['industry_type'] = df['categories'].apply(lambda x: [category['title'] for category in x])
df['industry_type'] = df['categories'].apply(lambda x: ', '.join(f'"{category["title"]}"' for category in x))

df.drop(columns=['categories'], inplace=True)

df

,id,name,coordinates,zip_code,industry_type
0,rWBOXmdnFTnTWc1jBne0Rw,Southern N Smoked,"39.276829, -76.613247",21230,"""Barbeque"", ""Smokehouse"", ""Soul Food"""
1,6ulR8s_SK3X0wYVQNZ0DSQ,Next Phaze Cafe,"39.291315, -76.613020",21202,"""Cafes"", ""Southern"", ""Seafood"""
2,s0LPuaTzR9euWzNT52o49Q,Sugarbee's Cafe & Grill,"39.291817, -76.615902",21201,"""American"", ""Wraps"", ""Caribbean"""
3,kTOlqpQmnA-8nWxE0lbFEA,The Land of Kush,"39.300167, -76.621694",21201,"""Vegan"", ""Soul Food"", ""Juice Bars & Smoothies"""
4,_-kMik32dINn2Xas6YvRzA,The Ivy Hotel,"39.303517, -76.612873",21202,"""Hotels"""
...,...,...,...,...,...
11643,pdR23jZYY4dsmFOrs0SnsA,Abeniskitchen,"39.398515, -76.795188",21117,"""Caterers"""
11651,lIMrkUAu20ukKbpmXOrWxg,JM'Aesthetics,"39.183771, -76.761217",21075,"""Eyelash Service"", ""Eyebrow Services"", ""Perman..."
11658,ip73G4r_HbQpoNUsOdEVLA,Southern Blues Soul Food,"39.369692, -76.798548",21133,"""Diners"""
13598,VefggQnaIJAd0u5bsMoYkw,Braids By Mirra,"39.434661, -76.779366",21117,"""Hair Stylists"""


In [200]:
# remove duplicate rows based on all columns, ie., drop all rows with the same values for each attribute
df = df.drop_duplicates()

# reset index after dropping duplicates
df.reset_index(drop=True, inplace=True)

df

,id,name,coordinates,zip_code,industry_type
0,rWBOXmdnFTnTWc1jBne0Rw,Southern N Smoked,"39.276829, -76.613247",21230,"""Barbeque"", ""Smokehouse"", ""Soul Food"""
1,6ulR8s_SK3X0wYVQNZ0DSQ,Next Phaze Cafe,"39.291315, -76.613020",21202,"""Cafes"", ""Southern"", ""Seafood"""
2,s0LPuaTzR9euWzNT52o49Q,Sugarbee's Cafe & Grill,"39.291817, -76.615902",21201,"""American"", ""Wraps"", ""Caribbean"""
3,kTOlqpQmnA-8nWxE0lbFEA,The Land of Kush,"39.300167, -76.621694",21201,"""Vegan"", ""Soul Food"", ""Juice Bars & Smoothies"""
4,_-kMik32dINn2Xas6YvRzA,The Ivy Hotel,"39.303517, -76.612873",21202,"""Hotels"""
...,...,...,...,...,...
1182,pdR23jZYY4dsmFOrs0SnsA,Abeniskitchen,"39.398515, -76.795188",21117,"""Caterers"""
1183,lIMrkUAu20ukKbpmXOrWxg,JM'Aesthetics,"39.183771, -76.761217",21075,"""Eyelash Service"", ""Eyebrow Services"", ""Perman..."
1184,ip73G4r_HbQpoNUsOdEVLA,Southern Blues Soul Food,"39.369692, -76.798548",21133,"""Diners"""
1185,VefggQnaIJAd0u5bsMoYkw,Braids By Mirra,"39.434661, -76.779366",21117,"""Hair Stylists"""


In [201]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187 entries, 0 to 1186
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             1187 non-null   object
 1   name           1187 non-null   object
 2   coordinates    1187 non-null   object
 3   zip_code       1187 non-null   object
 4   industry_type  1187 non-null   object
dtypes: object(5)
memory usage: 46.5+ KB


In [202]:
industries = df['industry_type'].unique()

# remove extra quotes and split industry types
cleaned_industries = [industry.strip('"') for group in industries for industry in group.split(', ')]

# print(cleaned_industries)


# find unique industry types in the list
unique_industries_unique = list(set(cleaned_industries))

#display the unique industry types
print(unique_industries_unique)
len(unique_industries_unique)

['Screen Printing/T-Shirt Printing', 'Pawn Shops', 'Investing', 'Art Space Rentals', 'Car Window Tinting', 'Pest Control', 'Cosmetology Schools', 'Chiropractors', 'Recycling Center', 'Vape Shops', 'Self Storage', 'Video Game Stores', 'TV Mounting', 'Caterers', 'Mobile Phone Repair', 'Air Duct Cleaning', 'Candy Stores', 'Electronics', 'Wraps', 'Personal Assistants', 'Private Tutors', 'Karaoke', 'Funeral Services & Cemeteries', 'Real Estate Services', 'Furniture Assembly', 'Demolition Services', 'Vehicle Wraps', 'Fences & Gates', 'Grocery', 'Hookah Bars', 'Customized Merchandise', 'Auto Detailing', 'DJs', 'Roadside Assistance', 'Painters', 'Biohazard Cleanup', 'Lighting Fixtures & Equipment', 'Food Stands', 'Eyelash Service', 'Caribbean', 'Vegan', 'Health Coach', 'Acai Bowls', 'Art Classes', 'Career Counseling', 'Silent Disco', 'Tree Services', 'Commissioned Artists', 'Session Photography', 'Office Equipment', 'Magicians', 'Damage Restoration', 'Criminal Defense Law', 'Puerto Rican', 'Ae

465

In [203]:
df

,id,name,coordinates,zip_code,industry_type
0,rWBOXmdnFTnTWc1jBne0Rw,Southern N Smoked,"39.276829, -76.613247",21230,"""Barbeque"", ""Smokehouse"", ""Soul Food"""
1,6ulR8s_SK3X0wYVQNZ0DSQ,Next Phaze Cafe,"39.291315, -76.613020",21202,"""Cafes"", ""Southern"", ""Seafood"""
2,s0LPuaTzR9euWzNT52o49Q,Sugarbee's Cafe & Grill,"39.291817, -76.615902",21201,"""American"", ""Wraps"", ""Caribbean"""
3,kTOlqpQmnA-8nWxE0lbFEA,The Land of Kush,"39.300167, -76.621694",21201,"""Vegan"", ""Soul Food"", ""Juice Bars & Smoothies"""
4,_-kMik32dINn2Xas6YvRzA,The Ivy Hotel,"39.303517, -76.612873",21202,"""Hotels"""
...,...,...,...,...,...
1182,pdR23jZYY4dsmFOrs0SnsA,Abeniskitchen,"39.398515, -76.795188",21117,"""Caterers"""
1183,lIMrkUAu20ukKbpmXOrWxg,JM'Aesthetics,"39.183771, -76.761217",21075,"""Eyelash Service"", ""Eyebrow Services"", ""Perman..."
1184,ip73G4r_HbQpoNUsOdEVLA,Southern Blues Soul Food,"39.369692, -76.798548",21133,"""Diners"""
1185,VefggQnaIJAd0u5bsMoYkw,Braids By Mirra,"39.434661, -76.779366",21117,"""Hair Stylists"""


In [204]:
# function to map businesses to a general industry based on industry_type column
def extract_general_industry(industry_types):
    industries = [industry.strip('"').lower() for industry in industry_types.split(', ')]
    
    # keywords for each general industry
    general_categories = {
        'Food & Dining': ['barbeque', 'smokehouse', 'food', 'cafes', 'southern', 'seafood', 'american', 'vegan', 'comfort food', 'italian', 'ice cream & frozen yogurt', 'waffles', 'chicken wings', 'mediterranean', 'kebab', 'diner', 'coffee', 'tea', 'bar', 'brunch', 'plate', 'sandwich', 'salad', 'pasta', 'dessert', 'bakeries', 'caribbean', 'empanadas', 'puerto rican', 'african', 'asian fusion', 'noodles', 'ramen'],
        'Health & Wellness': ['acupuncture', 'assisted living facilities', 'family practice', 'hospice', 'cosmetic dentists', 'occupational therapy', 'pediatric dentists', 'weight loss centers', 'eye', 'skin care', 'waxing', 'teeth whitening', 'reiki', 'acne treatment', 'pain management', 'mental health counseling', 'nutritionists', 'alternative medicine', 'naturopathic/holistic', 'psychologists', 'psychiatrists', 'life coach', 'counseling & mental health', 'trainers', 'chiropractors'],
        'Personal Care': ['spa', 'hair', 'nail', 'salon', 'men\'s hair salons', 'hair salons', 'cosmetology schools', 'nail technicians', 'massage therapy', 'massage', 'tattoo', 'hair removal', 'permanent makeup', 'makeup artists', 'bridal', 'sewing & alterations', 'piercing', 'estheticians', 'lash service', 'colonics', 'body contouring'],
        'Retail & Shopping': ['head shops', 'jewelry', 'bookstores', 'thrift stores', 'cosmetics & beauty supply', 'antiques', 'wigs', 'shoe stores', "store", 'clothing', 'accessories', 'florists', 'floral designers', 'gift shops', 'magazines', 'music & video', 'tobacco shops', 'shops'],
        'Professional Services': ['software development', 'advertising', 'marketing', 'graphic design', 'legal services', 'accountants', 'editorial services', 'real estate services', 'security services', 'caterers', 'cleaning', 'property management', 'business consulting', 'employment agencies', 'financial advising', 'debt relief services', 'notaries', 'process servers', 'tax services', 'investment firms', 'bankruptcy law', 'immigration law', 'business financing', 'talent agencies', 'real estate photography', 'video/film production', 'videographers', 'session photography', 'event photography', 'djs', 'commissioned artists', 'laboratory testing', 'criminal defense law', 'environmental testing', 'medical cannabis referrals', 'phlebologists', 'obstetricians & gynecologists', 'real estate agents', 'financial planning', 'investing', 'billing services', 'services'],
        'Education & Training': ['child care & day care', 'cosmetology schools', 'educational services', 'tutoring centers', 'tutor', 'specialty schools', 'test preparation', 'college counseling', 'dance studios', 'classes', 'school'],
        'Entertainment & Recreation': ['arts & crafts', 'recreation centers', 'psychics', 'party equipment rentals', 'karaoke', 'arcades', 'paint & sip', 'bowling', 'venues & event spaces', 'magicians', 'speakeasies', 'kids activities', 'bounce house rentals', 'cinema', 'zoo', 'museums', 'community service/non-profit', 'churches', 'social clubs', 'indoor playcentre', 'beer, wine & spirits', 'pubs', 'beer, wine & spirits', 'photo booth rentals', 'party supplies', 'party characters', 'silent disco', 'entertainment', 'musicians'],
        'Automotive Services': ['car buyers', 'oil change stations', 'auto repair', 'diy auto shop', 'car dealers', 'auto detailing', 'car rental', 'car wash', 'body shops', 'tires', 'towing', 'roadside assistance'],
        'Technology & IT': ['software development', 'mobile phones', 'web design', 'electronics', 'computer repair', 'data recovery', 'networking', 'cybersecurity', 'mobile phone repair'],
        'Finance & Insurance': ['mortgage lenders', 'insurance', 'tax services', 'investment firms', 'financial planning', 'accounting', 'credit unions', 'insurance brokers', 'check cashing', 'loans'],
        'Construction & Home Improvement': ['plumbing', 'electricians', 'carpentry', 'roofing', 'landscaping', 'painting', 'painters' 'flooring', 'home remodeling', 'handyman', 'carpenters', 'demolition', 'pressure washers', 'masonry/concrete', 'windows installation', 'decks & railing', 'fences & gates', 'installation', 'repair', 'wallpapering', 'signmaking', 'junk removal', 'movers', 'home'],
        'Travel & Hospitality': ['hotels', 'travel agencies', 'restaurants', 'tour operators', 'airlines', 'cruises', 'bed & breakfast', 'hostels', 'bike tours', 'bike rentals', 'limos', 'airport shuttles', 'boat charters', 'taxis', 'car share services', 'public transportation', 'town car service', 'travel services'],
        'Arts & Culture': ['art galleries', 'museums', 'theaters', 'concert halls', 'opera houses', 'dance companies', 'literary organizations', 'film festivals', 'art space rentals'],
        'Fitness & Sports': ['gyms', 'yoga studios', 'personal trainers', 'sports clubs', 'fitness equipment', 'athletic apparel', 'sports medicine', 'outdoor recreation', 'martial arts', 'boxing', 'self-defense', 'soccer', 'cardio classes', 'cycling classes', 'boot camps', 'yoga'],
        'Pets & Animals': ['pet grooming', 'veterinary clinics', 'pet supplies', 'dog training', 'animal shelters', 'pet sitting', 'dog walking', 'pet photography', 'pet groomers']
    }

    
    # count occurrences of keywords in industry types
    counts = {category: sum(any(keyword in industry for industry in industries) for keyword in keywords) 
              for category, keywords in general_categories.items()}
    
    # assign general industry based on keyword counts
    max_count = max(counts.values())
    if max_count == 0:
        return 'Other' #maybe change to "Miscelaneous" later
    else:
        for category, count in counts.items():
            if count == max_count:
                return category



df['general_industry'] = df['industry_type'].apply(extract_general_industry)

In [205]:
# the dataset has records of business outside of baltimore city, this filters those out

# reminder: on a new session, for this to work, run scraper.ipynb first
%store -r zip_codes
print(zip_codes)


df_filtered = df[df['zip_code'].isin(zip_codes)]
df_filtered

['21201', '21202', '21203', '21205', '21206', '21209', '21210', '21211', '21212', '21213', '21214', '21215', '21216', '21217', '21218', '21223', '21224', '21225', '21229', '21230', '21231', '21233', '21239', '21270', '21279', '21281', '21297']


,id,name,coordinates,zip_code,industry_type,general_industry
0,rWBOXmdnFTnTWc1jBne0Rw,Southern N Smoked,"39.276829, -76.613247",21230,"""Barbeque"", ""Smokehouse"", ""Soul Food""",Food & Dining
1,6ulR8s_SK3X0wYVQNZ0DSQ,Next Phaze Cafe,"39.291315, -76.613020",21202,"""Cafes"", ""Southern"", ""Seafood""",Food & Dining
2,s0LPuaTzR9euWzNT52o49Q,Sugarbee's Cafe & Grill,"39.291817, -76.615902",21201,"""American"", ""Wraps"", ""Caribbean""",Food & Dining
3,kTOlqpQmnA-8nWxE0lbFEA,The Land of Kush,"39.300167, -76.621694",21201,"""Vegan"", ""Soul Food"", ""Juice Bars & Smoothies""",Food & Dining
4,_-kMik32dINn2Xas6YvRzA,The Ivy Hotel,"39.303517, -76.612873",21202,"""Hotels""",Travel & Hospitality
...,...,...,...,...,...,...
1049,1Lp1rd75Ur5AlvBH-7mITA,CS Mobile Carwash,"39.230263, -76.614838",21225,"""Car Dealers""",Automotive Services
1051,-ROvITW-OloIV41WnAykKw,The BBQ Crew,"39.222570, -76.628210",21225,"""Barbeque""",Food & Dining
1069,16Di256ooYFTWjgQQWod4A,Cash Loans Bear,"39.217941, -76.615307",21225,"""Check Cashing/Pay-day Loans"", ""Installment Lo...",Finance & Insurance
1071,gqWgDq8zQOxBI0WqhRQPlA,JahFast Power Washing,"39.217007, -76.617392",21225,"""Pressure Washers""",Construction & Home Improvement


# Save to file

In [206]:
df_filtered.to_csv('..\\OutputFiles\\yelp_processed_data.csv', index=False)